In [ ]:
import random
random.seed(10)

In [ ]:
import re
import math
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from random import *

# **Loading Text**

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

import os

path = '/content/drive/MyDrive/Colab Notebooks/Cap11'

Mounted at /content/drive


In [ ]:
text = open(os.path.join(path, 'texto.txt'), 'r').read()

# **Data PreProcessing**

In [ ]:
sentences = re.sub('[,.!?\\-]', '', text.lower()).split('\n')

In [ ]:
print(sentences)

["'olá como vai eu sou a camila\\n'", "'olá camila meu nome é fernando muito prazer\\n'", "'prazer em conhecer você também como você está hoje\\n'", "'ótimo meu time de futebol venceu a competição\\n'", "'uau parabéns fernando\\n'", "'obrigado camila\\n'", "'vamos comer uma pizza mais tarde para celebrar\\n'", "'claro você recomenda algum restaurante camila\\n'", "'sim abriu um restaurante novo e dizem que a pizza de banana é fenomenal\\n'", "'ok nos encontramos no restaurante às sete da noite pode ser\\n'", "'pode sim nos vemos mais tarde então'"]


In [ ]:
word_list = list(set(' '.join(sentences).split()))

In [ ]:
print(word_list)

["fernando\\n'", 'muito', "'olá", "'vamos", 'tarde', "hoje\\n'", "'prazer", 'venceu', "prazer\\n'", 'algum', "competição\\n'", 'conhecer', "'uau", "'ok", "celebrar\\n'", "fenomenal\\n'", 'novo', 'pode', 'restaurante', 'para', 'da', 'é', 'no', 'time', 'a', 'camila', 'um', 'encontramos', 'de', 'está', 'meu', 'pizza', 'em', "'sim", 'mais', 'abriu', 'e', 'você', 'vemos', "'ótimo", "ser\\n'", 'também', 'vai', 'parabéns', 'sim', "então'", 'recomenda', 'que', 'noite', 'nome', "'claro", 'fernando', 'eu', 'sou', 'comer', 'uma', "camila\\n'", 'sete', 'nos', 'às', 'dizem', 'banana', 'futebol', "'pode", 'como', "'obrigado"]


In [ ]:
word_dict = {'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3}

In [ ]:
word_dict

{'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3}

In [ ]:
for i, w in enumerate(word_list):
    word_dict[w] = i + 4

In [ ]:
print(word_dict)

{'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3, "fernando\\n'": 4, 'muito': 5, "'olá": 6, "'vamos": 7, 'tarde': 8, "hoje\\n'": 9, "'prazer": 10, 'venceu': 11, "prazer\\n'": 12, 'algum': 13, "competição\\n'": 14, 'conhecer': 15, "'uau": 16, "'ok": 17, "celebrar\\n'": 18, "fenomenal\\n'": 19, 'novo': 20, 'pode': 21, 'restaurante': 22, 'para': 23, 'da': 24, 'é': 25, 'no': 26, 'time': 27, 'a': 28, 'camila': 29, 'um': 30, 'encontramos': 31, 'de': 32, 'está': 33, 'meu': 34, 'pizza': 35, 'em': 36, "'sim": 37, 'mais': 38, 'abriu': 39, 'e': 40, 'você': 41, 'vemos': 42, "'ótimo": 43, "ser\\n'": 44, 'também': 45, 'vai': 46, 'parabéns': 47, 'sim': 48, "então'": 49, 'recomenda': 50, 'que': 51, 'noite': 52, 'nome': 53, "'claro": 54, 'fernando': 55, 'eu': 56, 'sou': 57, 'comer': 58, 'uma': 59, "camila\\n'": 60, 'sete': 61, 'nos': 62, 'às': 63, 'dizem': 64, 'banana': 65, 'futebol': 66, "'pode": 67, 'como': 68, "'obrigado": 69}


In [ ]:
number_dict = {i: w for i, w in enumerate(word_dict)}

In [ ]:
print(number_dict)

{0: '[PAD]', 1: '[CLS]', 2: '[SEP]', 3: '[MASK]', 4: "fernando\\n'", 5: 'muito', 6: "'olá", 7: "'vamos", 8: 'tarde', 9: "hoje\\n'", 10: "'prazer", 11: 'venceu', 12: "prazer\\n'", 13: 'algum', 14: "competição\\n'", 15: 'conhecer', 16: "'uau", 17: "'ok", 18: "celebrar\\n'", 19: "fenomenal\\n'", 20: 'novo', 21: 'pode', 22: 'restaurante', 23: 'para', 24: 'da', 25: 'é', 26: 'no', 27: 'time', 28: 'a', 29: 'camila', 30: 'um', 31: 'encontramos', 32: 'de', 33: 'está', 34: 'meu', 35: 'pizza', 36: 'em', 37: "'sim", 38: 'mais', 39: 'abriu', 40: 'e', 41: 'você', 42: 'vemos', 43: "'ótimo", 44: "ser\\n'", 45: 'também', 46: 'vai', 47: 'parabéns', 48: 'sim', 49: "então'", 50: 'recomenda', 51: 'que', 52: 'noite', 53: 'nome', 54: "'claro", 55: 'fernando', 56: 'eu', 57: 'sou', 58: 'comer', 59: 'uma', 60: "camila\\n'", 61: 'sete', 62: 'nos', 63: 'às', 64: 'dizem', 65: 'banana', 66: 'futebol', 67: "'pode", 68: 'como', 69: "'obrigado"}


In [ ]:
vocab_size = len(word_dict)
print(vocab_size)

70


In [ ]:
token_list = list()

In [ ]:
for sentence in sentences:
  arr = [word_dict[s] for s in sentence.split()]
  token_list.append(arr)

In [ ]:
text[0:29]

"'Olá, como vai? Eu sou a Cami"

In [ ]:
token_list[0]

[6, 68, 46, 56, 57, 28, 60]

# **Hyperparameters**

In [ ]:
batch_size = 6
n_segments = 2
dropout = 0.2

max_len = 100

max_pred = 7

n_layers = 6

n_heads = 12

d_model = 768

d_ff = d_model * 4

d_k = d_v = 64

NUM_EPOCHS = 50